In [1]:
#Training data augmentation
#Written by Lehan Yao from Qian Chen Group, MatSE, UIUC
#Sept. 21, 2021

In [3]:
import cv2
import numpy as np
import glob
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
labelPath = 'manual label'
nameList = [os.path.basename(x) for x in glob.glob(labelPath+'/image/*.tif')]
stack = np.zeros([len(nameList),512,512,3]) #container for storing the labels and images

i = 0
for name in nameList: #read the labels and images and store them in RGB channels
    I_image = cv2.imread(labelPath + '/image/' + name )
    I_particle = cv2.imread(labelPath + '/mask/' + name )[:,:,2]
    I_polymer = cv2.imread(labelPath + '/mask/' + name )[:,:,1]
    stack[i,:,:,0] = cv2.cvtColor(I_image, cv2.COLOR_BGR2GRAY)
    stack[i,:,:,1] = I_particle
    stack[i,:,:,2] = I_polymer
    i += 1

In [8]:
volume = 1200 #the number of images in the augmented dataset
data_gen_args_train = dict(fill_mode = 'reflect', rotation_range=360, zoom_range=0.1, horizontal_flip = True, vertical_flip = True, brightness_range = (0.8,1.2))
datagen_train = ImageDataGenerator(**data_gen_args_train)
seed = 1
image_generator_train = datagen_train.flow(stack ,seed = seed) #RGB channels thus labels and images will be transformed together
i = 0
no = 0
while (no < volume):
    im = next(image_generator_train)
    for j in range(im.shape[0]):
        t = im[j,:,:,0]
        m1 = im[j,:,:,1] > 128
        m2 = im[j,:,:,2] > 128
        m3 = np.logical_not(np.logical_or(m1,m2)) #m3 is the background class
        m = np.stack([m1,m2,m3],2)
        t = np.uint8(t)
        m = np.uint8(m)*255
        cv2.imwrite('train/'+str(no)+'.tif', t)
        cv2.imwrite('label/'+str(no)+'.tif', m)
        no += 1
        if not no < volume:
            break
    i += 1
